# Decision Tree

In [1]:
import sys
sys.path.insert(1, '../../libs')
from utils import get_data, temporal_train_test_split
from autorship import AuthorClassifier
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import TweetTokenizer
import numpy as np
import pandas as pd

In [2]:
data = get_data("../../data/authors.csv")

In [3]:
tt = TweetTokenizer()

vectorizers = [CountVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,5), analyzer="char"),
                CountVectorizer(ngram_range=(4,5), analyzer="char"),
                CountVectorizer(ngram_range=(3,8), analyzer="char"),
                TfidfVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(4,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(3,8), analyzer="char")]

In [18]:
from joblib import Parallel, delayed
def process(vectorizer):
    evaluation = list()
    usernames = list(np.unique(data["username"]))
    
    vectorizer_str = vectorizer.__str__()
    print(f"Running => {vectorizer_str}")

    for i in range(len(usernames)):
        author1 = usernames.pop()

        for author2 in usernames:
            X_train, X_test, y_train, y_test = temporal_train_test_split(
                data, author1, author2)

            clf = AuthorClassifier(vectorizer, clf=GradientBoostingClassifier(random_state=42), scaler=MaxAbsScaler())
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            evaluation.append(clf.evaluate(y_test, y_pred))
            metrics = pd.DataFrame(evaluation)[["f1_macro", "recall_macro", "precision_macro", "accuracy", "auc_score"]].mean()
            
    return (vectorizer_str, metrics)
    
results = Parallel(n_jobs=8)(delayed(process)(vectorizer) for vectorizer in vectorizers)
print(results) 

Running => CountVectorizer(analyzer='char', ngram_range=(1, 5))
Running => CountVectorizer(analyzer='char', ngram_range=(4, 5))
Running => TfidfVectorizer(analyzer='char', ngram_range=(1, 5))
Running => CountVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f2651271cd0>>)
Running => CountVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f816843fc70>>)
Running => TfidfVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f8d998facd0>>)
Running => CountVectorizer(analyzer='char', ngram_range=(3, 8))
Running => TfidfVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fdb07197cd0>>)
Running => TfidfVectorizer(analyzer='char', ngram_range=(4, 5))
Running => TfidfVectorizer(analyzer=

In [5]:
metrics_df = pd.DataFrame([results[i][1] for i in range(len(results))])
metrics_df["vectorizer"] = [results[i][0] for i in range(len(results))]
metrics_df.to_csv("../../results/gradient_boosting.csv")
metrics_df

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.822999,0.823577,0.824949,0.823258,0.823567,CountVectorizer(tokenizer=<bound method TweetT...
1,0.826782,0.827400,0.829040,0.827100,0.827390,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.807534,0.807786,0.808186,0.807667,0.807786,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.737161,0.738153,0.740325,0.737943,0.745735,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.741921,0.742537,0.743875,0.742375,0.748380,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.807090,0.807593,0.808530,0.807306,0.807577,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.791719,0.792471,0.794454,0.792313,0.792467,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.794094,0.794336,0.794823,0.794297,0.794336,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.721641,0.722910,0.725448,0.722808,0.729435,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.716958,0.717963,0.720158,0.717941,0.723385,"TfidfVectorizer(analyzer='char', ngram_range=(..."


Row with max sum of columns (remember that by the result of f1-score we use the precision and recall score)

In [6]:
print(metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()])
print("\nVectorizer:", metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()]["vectorizer"])

f1_macro                                                    0.826782
recall_macro                                                  0.8274
precision_macro                                              0.82904
accuracy                                                      0.8271
auc_score                                                    0.82739
vectorizer         CountVectorizer(ngram_range=(1, 3),\n         ...
Name: 1, dtype: object

Vectorizer: CountVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fc089011ca0>>)


In [7]:
pd.read_csv("../../results/gradient_boosting.csv").drop("Unnamed: 0", axis=1)

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.822999,0.823577,0.824949,0.823258,0.823567,CountVectorizer(tokenizer=<bound method TweetT...
1,0.826782,0.827400,0.829040,0.827100,0.827390,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.807534,0.807786,0.808186,0.807667,0.807786,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.737161,0.738153,0.740325,0.737943,0.745735,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.741921,0.742537,0.743875,0.742375,0.748380,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.807090,0.807593,0.808530,0.807306,0.807577,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.791719,0.792471,0.794454,0.792313,0.792467,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.794094,0.794336,0.794823,0.794297,0.794336,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.721641,0.722910,0.725448,0.722808,0.729435,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.716958,0.717963,0.720158,0.717941,0.723385,"TfidfVectorizer(analyzer='char', ngram_range=(..."
